# Project 1 – Decision Trees and Random Forests

In [94]:
# Reload all modules without having to restart the kernel
# Useful for development if you have edited any of the external code files.
%load_ext autoreload
%autoreload 2

# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from random_forest import RandomForest
from decision_tree import DecisionTree

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
seed = 42
np.random.seed(seed)

## Dataset

Do data loading, exploration and preprocessing as you see fit.

Here is some code to load the dataset to get you started.

In [96]:
data = np.genfromtxt("letters.csv", delimiter=",", dtype=float, names=True)

feature_names = list(data.dtype.names[:-1])
target_name = data.dtype.names[-1]

X = np.array([data[feature] for feature in feature_names]).T
y = data[target_name].astype(int)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.3, random_state=seed, stratify=y
)

print(f"Feature columns names: {feature_names}")
print(f"Target column name: {target_name}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Feature columns names: ['xbox', 'ybox', 'width', 'high', 'onpix', 'xbar', 'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'xege', 'xegvy', 'yege', 'yegvx']
Target column name: label
X shape: (2000, 16)
y shape: (2000,)


In [ ]:
param_grid_dt = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 50, 100],
}

# #dt = DecisionTreeClassifier(random_state=0)
# dt = DecisionTree()

# grid_dt = GridSearchCV(
#     estimator=dt,
#     param_grid=param_grid_dt,
#     cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=seed),
#     scoring="accuracy",
#     n_jobs=-1,
# )

# grid_dt.fit(X_trainval, y_trainval)

# print("Best DT hyperparameters:", grid_dt.best_params_)
# print("Best mean CV accuracy:", grid_dt.best_score_)

# test_preds = grid_dt.best_estimator_.predict(X_test)
# print("Test accuracy:", accuracy_score(y_test, test_preds))

print("Best DT hyperparameters:", grid_dt.best_params_)
print("Best mean CV accuracy:", grid_dt.best_score_)

# Evaluate on unseen test set
test_preds = grid_dt.best_estimator_.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, test_preds))


In [101]:
param_grid_rf = {
    "n_estimators": [2, 5, 10, 15, 20, 50],
    "criterion": ["gini", "entropy"],
    "max_depth": [2, 5, 10, 20, 50, None],
    "max_features": ["sqrt", "log2", None]
}

# rf_wrapper = RandomForest()

# grid_rf = GridSearchCV(
#     estimator=rf_wrapper,
#     param_grid=param_grid_rf,
#     cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=seed),
#     scoring="accuracy",
#     n_jobs=-1
# )

# grid_rf.fit(X_trainval, y_trainval)

# print("Best RF hyperparameters:", grid_rf.best_params_)
# print("Best mean CV accuracy:", grid_rf.best_score_)

print("Best RF hyperparameters:", grid_rf.best_params_)
print("Best mean CV accuracy:", grid_rf.best_score_)

# Evaluate on unseen test set
test_preds = grid_rf.best_estimator_.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, test_preds))



ValueError: 
All the 1080 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/INF264/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hannarasmussen/Documents/Documents/3 Studier/7.semester/INF264-Project-1-H25/random_forest.py", line 48, in fit
NameError: name 'seed' is not defined


In [ ]:
result = permutation_importance(
    estimator=grid_rf.best_estimator_,
    X=X_test,
    y=y_test,
    scoring="accuracy",
    n_repeats=30,
    random_state=42,
    n_jobs=-1 
)

print("Feature importances:", result.importances_mean)

# # Plot
# feature_names = [f"feature {i}" for i in range(X_test.shape[1])]

# feature_names = [f"feature {i}" for i in range(X_test.shape[1])]
# plt.barh(feature_names, result.importances_mean)
# plt.xlabel("Mean accuracy decrease")
# plt.title("Permutation Importance (Random Forest)")
# plt.show()

df = pd.read_csv("letters.csv")
feature_names = df.columns[:-1]
X = df.drop("label", axis=1).values
y = df["label"].values

def plot_permutation_importance(importances, feature_names):
    plt.figure(figsize=(10, 6))
    sorted_idx = np.argsort(importances)[::-1]
    plt.bar(range(len(importances)), importances[sorted_idx], align="center")
    plt.xticks(range(len(importances)), np.array(feature_names)[sorted_idx], rotation=90)
    plt.title("Permutation Importance (Random Forest)")
    plt.tight_layout()
    plt.show()

plot_permutation_importance
